## Take a existing microtonal piece of mine and use the algorithmic tools to explore it

This notebook takes a piece I wrote in 2018:  Balloon Drum Music for Piano, which itself is a modification of a piece called Balloon Drum Music from 2005. The structure of the piece is a verse in B♭ major, followed by a bridge of interesting chord changes, then back to the B♭ major. 

Here is the structure with the Sagittal① notation for the notes.

<img src='http://ripnread.com/listen/BalloonDrumKeys.jpg' >

This version uses the tonality diamond to the 31-limit, constructed by calculating the overtones (referred to as otonalities) up to number 31, then the undertones (referred to as utonalities) on each of those overtones. There are 16 scales of 16 notes each, either 16 otonal scales on the 16 utonalities, or 16 utonal scales on the 16 otonalities. When you eliminate duplicate notes, you end up with 214 scale degrees per octave. I added a few additional ones near the top and bottom to fill in large gaps between notes, and ended up with 217 notes per octave. 

It an extension of the work of Harry Partch and his Tonality Diamond, which went to the 11-limit. 

I started my microtonal journey with the Partch 11-limit diamond. I later explored higher dimensions by using the 15-limit, which extends the over/under tones to the 15th overtone. The 15th overtone on C is B♮, which is convenient for some modulations. It also includes the 13th overtone, a challenging interval in the same way that the 11th overtone in Partch's 11-limit diamond can be jarring to the ear.  I hoped to discover more unique sounds in the upper and lower regions of the diamond by extending it to the next logical step: 31-limit. As Partch once said, "There's a whole world between the 1:1 and the 2:1". Or he might not have said that. Who knows?

There is a more complete explanation of the tuning here: __[here](http://ripnread.com/partch-tonality-diamond-to-the-31-limit/)__.

My diamond image in that link has the Sagittal① symbols and rations for each of the notes. 

__[Tonality Diamond to the 31-limit](http://ripnread.com/listen/Diamond_31-limit.jpg)__

Below is an image of what I've always referred to as the Cassandra version of the diamond, which is more compact than the diamond.
The otonal scales read from left to right, and the utonal scales read from top to bottom. So you have otonal on C in the upper left corner, then reading down, you get otonal on B-, A++, A♮, G+, G-, F++, F-, E♭, D++, D+, D-, D♭, & C+. The utonal scales start with the same C in the upper left corner, followed by reading to the right, you see Utonal on D♭, D+, D++, E-, F-, F♯, G♭, G♮, G++, A♭, A+, A♯, B♭, B-, & B++. 

① Sagittal is a font designed for helping musicians more easily visualize the tuning in musical scores. More about that __[here](https://sagittal.org/)__.

<img src='http://ripnread.com/listen/31-limit-diamond.jpg'>

Jupyter and python don't support variable names using Sagittal fonts, so I use close approximations in ASCII:
<code> Dpmaj_A_3, G_min_A_3, Dbmaj_A_4, Fnmin_A_2, C_maj_A_1, Ebmin_A_1, Bbmaj_A_2, Emmin_A_1, Bbmaj_A_2</code>

You can listen to the original piece here:
__[2005 Original Balloon Drum Music](http://ripnread.com/balloon-drum-music/)__

And a later version for my microtonal slide Bosendorfer piano here:
__[2018 Balloon Drum Music for Piano](http://ripnread.com/balloon-drum-music-19/)__

In [59]:
import numpy as np
import numpy.testing as npt
import copy
import mido
import time
from importlib import reload
from midi2audio import FluidSynth
from IPython.display import Audio, display
import music21 as m
import os
import muspy
import pandas as pd
import sys
sys.path.insert(0, '/home/prent/Dropbox/Tutorials/coconet-pytorch/coconet-pytorch-csound')
import piano as p
import selective_stretching_codes as stretch
import samples_used as su
import subprocess
from numpy.random import default_rng
rng = np.random.default_rng()
soundfont = '../coconet-pytorch/font.sf2' # you will need to download this from location specified in the github README.md
midi_dir = 'eval'
CSD_FILE = 'ball4.csd'
LOGNAME = 'ball4.log'
WAV_OUT = '/home/prent/Music/sflib/ball4.wav'
CON_OUT = '/home/prent/Music/sflib/ball4-t'
downbeat = 1 # all the synthetic chorales out of TonicNet have a downbeat of 1

OCTAVE_SIZE = 217 # there are 217 distinct pictches per octave in the tonality diamond to the 31-limit

In [60]:
# for each of the 8 keys, build the chords from the scales created above.
# key is a string index into the keys dictionary: e.g. 'Bbmaj'
# inversion is the string index into the inversions dictionary e.g. 'A_oton_1'
# the inversion includes the rank is which of the four tetrachords in each scale, A, B, C, or D
# And it specifies which note is on top
# in

def build_chords(key, inversion):
    chord = np.array([keys[key][note] for note in (inversions[inversion])]).reshape(-1,1)
    return chord

In [61]:
# build the chord structures for the different modes for the 16 keys

# build the 16 tone scales in each of the 8 keys used in the piece
keys = {'Bbmaj': np.array([179, 200, 0, 16, 34, 49, 63, 77, 90, 103, 115, 127, 138, 147, 159, 169]), 
        'Dpmaj': np.array([42, 61, 79, 95, 112, 127, 141, 155, 168, 180, 194, 210, 0, 6,21, 33]), 
        'Dbmaj': np.array([19, 40, 58, 75, 90, 105, 120, 133, 146, 159, 172, 183, 196, 212, 0, 4]),
        'C_maj': np.array([0, 18, 38, 55, 71, 87, 99, 113, 127, 139, 151, 163, 175, 186, 198, 214]),
        'G_min': np.array([38, 17, 0, 201, 183, 168, 154, 140, 127, 114, 102, 90, 79, 70, 58, 48]),
        'F_min': np.array([0, 199, 179, 162, 146, 130, 118, 104, 90, 78, 66, 54, 42, 31, 19, 3]),
        'Ebmin': np.array([175, 156, 138, 122, 105, 90, 76, 62, 49, 37, 23, 7, 0, 211, 196, 184]),
        'Emmin': np.array([198, 177, 159, 142, 127, 112, 97, 84, 71, 58, 45, 34, 21, 5, 0, 213])}

# choose the cluster and the inversion for each chord
inversions = {'A_oton_1': np.array([0, 4, 8, 12]),
            'B_oton_1': np.array([2, 6, 10, 14]),
            'C_oton_1': np.array([1, 5, 9, 13]),
            'D_oton_1': np.array([3, 7, 11, 15]),
            'A_uton_1': np.flip(np.array([0, 4, 8, 12])),
            'B_uton_1': np.flip(np.array([2, 6, 10, 14])),
            'C_uton_1': np.flip(np.array([1, 5, 9, 13])),
            'D_uton_1': np.flip(np.array([3, 7, 11, 15])),
            'A_oton_2': np.array([4, 8, 12, 0]),
            'B_oton_2': np.array([6, 10, 14, 2]),
            'C_oton_2': np.array([5, 9, 13, 1]),
            'D_oton_2': np.array([7, 11, 15, 3]),
            'A_uton_2': np.flip(np.array([4, 8, 12, 0])),
            'B_uton_2': np.flip(np.array([6, 10, 14, 2])),
            'C_uton_2': np.flip(np.array([5, 9, 13, 1])),
            'D_uton_2': np.flip(np.array([7, 11, 15, 3])),
            'A_oton_3': np.array([8, 12, 0, 4]),
            'B_oton_3': np.array([10, 14, 2, 6]),
            'C_oton_3': np.array([9, 13, 1, 5]),
            'D_oton_3': np.array([11, 15, 3, 7]),
            'A_uton_3': np.flip(np.array([8, 12, 0, 4])),
            'B_uton_3': np.flip(np.array([10, 14, 2, 6])),
            'C_uton_3': np.flip(np.array([9, 13, 1, 5])),
            'D_uton_3': np.flip(np.array([11, 15, 3, 7])),
            'A_oton_4': np.array([12, 0, 4, 8]),
            'B_oton_4': np.array([14, 2, 6, 10]),
            'C_oton_4': np.array([13, 1, 5, 9]),
            'D_oton_4': np.array([15, 3, 7, 11]),
            'A_uton_4': np.flip(np.array([12, 0, 4, 8])),
            'B_uton_4': np.flip(np.array([14, 2, 6, 10])),
            'C_uton_4': np.flip(np.array([13, 1, 5, 9])),
            'D_uton_4': np.flip(np.array([15, 3, 7, 11]))}

In [83]:
# Build the data structures required for the csound processing
# voices to use in the orchestra:
# finger piano 1, bass fp 2, balloon drum low 3, med 4, high 5, bass flute 6, oboe 7, clarinet 8, bassoon 9, french horn 10
# 1    2   3   4    5   6   7   8    9    10
# fnp bfp bdl bdm  bdh bflt obo clar bass frn
#                 S   A   T   B   S   A   T   B   S   A   T   B
voices = 16 # how many voices for all the chorales - I've only tested 16, 32,& 64.
list_voices =    [1,  1,  1,  2,  3,  3,  4,  5,  1,  1,  1,  2]
list_velocity = [66, 65, 64, 64, 65, 66, 64, 65, 67, 66, 66, 65]
o_vol = 27 # overall volume

list_volume = [o_vol, o_vol, o_vol, o_vol, o_vol, o_vol, o_vol, o_vol, o_vol, o_vol, o_vol, o_vol]
list_holds = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
list_envs = [1, 0, 2, 8] 
list_envs = [list_envs, list_envs, list_envs, list_envs, list_envs, list_envs, 
             list_envs, list_envs, list_envs, list_envs, list_envs, list_envs]
low_p = 0.25 #  probability choice - rarely choose to hold the note, most events are stacatto
list_env_probs = [1 - (low_p * 3), low_p, low_p, low_p]
print(f'{list_env_probs = }')
all_env_probs = [list_env_probs, list_env_probs, list_env_probs, list_env_probs, list_env_probs, list_env_probs, 
                  list_env_probs, list_env_probs, list_env_probs, list_env_probs, list_env_probs, list_env_probs]
#                
voice_array = (list_voices, list_holds, list_volume, list_velocity, list_envs, all_env_probs)
i = 0
for variable in voice_array:
    assert len(variable) == len(voice_array[0])
    i += 1
for env, probs in zip(list_envs, all_env_probs):
    assert len(env) == len(probs)
npt.assert_almost_equal(np.sum(list_env_probs), 1.0,0.001)

list_env_probs = [0.25, 0.25, 0.25, 0.25]


In [76]:
# variables called for in the assembling of multiple chorales
# the first 15 are global to all the chorales
stretch.start_logging({'log': LOGNAME})

# number file name of the output audio files - consecutive helps you know what the most recent one was called
version = 19
# if you want to render the data as an audio file using csound, set csound to True
csound = True
# If you don't want csound to process the entire file, set limit to the number of seconds you would like rendered, or zero for no limit
limit = 0
# if you want csound to convolve the output file with an impulse response file, set convolv to true
convolv = True

# what tempo should be assigned to each chorale in the list. 
# the tempo is set to the lower of the elapsed time // 1.5 
# or a random tempo in a range of the next two variables
assigned_tempo_minimum = 90
assigned_tempo_maximum = 110

# end a chorale with a slow down repeating the last few steps a certain number of time, after the arpeggiation 
slow_time_steps = -7 # -8 means the last 8 time steps are repeated and transformed from 1/16th notes to 14 notes based on the next setting
slot_repeat = 4 # 2 means repeat the slow_time_steps by this many times. Repeat 4 transform every 1/16th note into a 1/4 note

# the following control the octave shifts. They are made in batches of notes, and the system attempts to do the increases so that 
# a given percentage of notes are increased or decreased by an octave. It's done twice, so some notes are moved two octaves
stretch_octaves_percent_up = [.35, .15] # what percent of notes will be raised an octave - 1st number is the 2nd increase, 2nd is the 2nd
stretch_octaves_percent_down = [.25, .25] # what percent of notes will be lowered an octave - 1st number is the 2nd decrease, 2nd is the 2nd
stretch_octaves_duration_up = [45, 25] # how long should each batch of increases be
stretch_octaves_duration_down = [20, 40]  # how long should each batch of decreases be

pause_between_chorales = 2
pad8 = np.zeros((voices,1),dtype = int) # add silence at the end of the chorale
pad16 =  np.zeros((voices, pause_between_chorales), dtype = int)

# If it gets large, you have to reduce ovol variable in the previous cell by quite a bit to avoid distortion in csound

# The following constants affect values that are determined in each chorale

# the following four variables control the mask used in the arpeggiation. More zeros = more sparse result. 
zeros_low = 7 # minimum number of zeros 
zeros_high = 12 # maximum number of zeros
ones_low = 2 # minimum number of ones
ones_high = 7 # maximum number of ones
mask_length_low = 16 # minimum length in time_steps for the mask before repeating
mask_length_high = 28 # maximum length for the mask before repeating

# repeat every time_step by this factor to slow down the chord changes. The higher the number the longer the chorale
# each chorale has it's own repeat settings up to 8. 
# Also really livens up the rhythm, each quarter note is a different length
repeat_count = rng.integers(low = 3, high = 10, size = repeat_sections) 

high_all_tile = 13
high_verse_repetitions = 6 # how many times to repeat the 4 verse sections. Up to this number of repetitions
# # the following control the degree of expansion of the challenging steps sections.
# # typical numbers for five chorales in under 18 minutes are 1, 8, 2 np.max(factors) = 6, np.min(factors) = 2, np.mean(factors) = 4.0
# # for very short single chorales, you can choose higher numbers, like 1, 20, 5 np.max(factors) = 15, np.min(factors) = 5, np.mean(factors) = 10.7 
# # 1, 13, 3 yeidlds np.max(factors) = 9, np.min(factors) = 3, np.mean(factors) = 6.18
# repeating_factors = [[1,13,4],[1,9,3],[1,8,2],[1,18,6],[1,18,5]]
# How many different sections will there be. Each is one of A, B, C, D
repeat_sections = 4 * 4 + 1
# How many different sections will there be. Each is one of A, B, C, D
repeat_sections = 1 # make it very short for learning how to combine winds with the percussion

logging to: ball4.log


In [77]:
def build_chorale (rank, short_bridge = False, long_verse = False):
    
    # go through the bridge chord changes
    if short_bridge:
        bridge = np.concatenate((build_chords("Dpmaj", rank + "_oton_3"), build_chords("G_min", rank + "_uton_3"), 
                         build_chords("Dbmaj", rank + "_oton_1"), build_chords("F_min", rank + "_uton_1"), 
                         build_chords("C_maj", rank + "_oton_1"), build_chords("Ebmin", rank + "_uton_4"), 
                         build_chords("Bbmaj", rank + "_oton_2")), axis = 1)
    else: bridge = np.concatenate((build_chords("Dpmaj", rank + "_oton_3"), build_chords("G_min", rank + "_uton_3"), 
                         build_chords("Dbmaj", rank + "_oton_1"), build_chords("F_min", rank + "_uton_1"), 
                         build_chords("C_maj", rank + "_oton_1"), build_chords("Ebmin", rank + "_uton_4"), 
                         build_chords("Bbmaj", rank + "_oton_2"), build_chords("Emmin", rank + "_uton_4"), 
                         build_chords("Bbmaj", rank + "_oton_2")), axis = 1)
    bridge_soprano = bridge + (4 * OCTAVE_SIZE) # take it up 4 octaves above the base octave 0
    bridge_bass = bridge +  (3 * OCTAVE_SIZE) # take it up 3 octaves above the bass octave
    bridge = np.concatenate((bridge_soprano, bridge_bass), axis = 0)
    
    # choose an arrangement of the four inversions for the verse
    verse = np.concatenate(rng.choice([build_chords("Bbmaj", rank + "_oton_1"), 
                                        build_chords("Bbmaj", rank + "_oton_2"), 
                                        build_chords("Bbmaj", rank + "_oton_3"), 
                                        build_chords("Bbmaj", rank + "_oton_4")], size = 4, replace = False, shuffle = True),
                                        axis = 1)
    if long_verse: 
        tile_verse = 2 * rng.integers(low = 3, high = high_verse_repetitions, size = None) # more verses than predicted
    else: 
        tile_verse = rng.integers(low = 3, high = high_verse_repetitions, size = None)
        p.logging.info(f'number of repetition of the verses before the bridge: {tile_time_steps = }')

    verse = np.tile(verse, tile_verse) # make many trips throught the verse inversions
    verse_soprano = verse + (4 * OCTAVE_SIZE)
    verse_bass = verse + (3 * OCTAVE_SIZE)
    verse = np.concatenate((verse_soprano, verse_bass), axis = 0)
    return (bridge, verse)

In [78]:
tempos = 't 0 '
elapsed_time = 0
new_tempo = 100
last_time_step = np.empty((repeat_sections),dtype = int)
concat_chorale = np.empty((voices,0),dtype = int)
for i in range(repeat_sections):
    p.logging.info(f'section #{i} out of {repeat_sections = }') 
    
    if i % 4 == 0:
        rank_type = 'A'
        long_verse = rng.choice([True, False], p = ([.7, .3])) # verse is twice as long on the 'A' rank 70% of the time
        short_bridge = rng.choice([True, False], p = ([.4, .6])) # bridge is short 40% of the time on the 'A' rank
    elif i % 4 == 1:
        rank_type = 'B'
        long_verse = rng.choice([True, False], p = ([.2, .8])) # verse is twice as long on the 'B' rank 20% of the time
        short_bridge = rng.choice([True, False], p = ([.8, .2])) # bridge is short 80% of the time on 'B' rank
    elif i % 4 == 2:
        rank_type = 'C'
        long_verse = rng.choice([True, False], p = ([.5, .5])) # verse is twice as long on the 'C' rank 50% of the time
        short_bridge = rng.choice([True, False], p = ([.2, .8])) # bridge is short 20% of the time on the 'C' rank
    elif i % 4 == 3:
        rank_type = 'D'
        long_verse = False
        short_bridge = rng.choice([True, False], p = ([.8, .2])) # bridge is short 80% of the time on 'D' rank
    
    tile_time_steps = rng.integers(low = 3, high = high_all_tile, size = None)
    
    p.logging.info(f'bridge and verse from rank {rank_type = }, {short_bridge = }') 
    # make the bridge and verse based on the rank. It might be shorter than normal 40% of the time
    bridge, verse = build_chorale(rank_type, short_bridge = short_bridge, long_verse = long_verse)
    p.logging.info(f'bridge and verse from rank {rank_type = }, {short_bridge = }') 
    p.logging.info(f'{bridge.shape = }, {verse.shape = }')
    chorale = np.concatenate((verse, bridge), axis = 1) # combine the bridge and the verse along the time_step axis
    p.logging.info(f'after combining bridge and verse: {chorale.shape = }')

    chorale = np.repeat(chorale, tile_time_steps, axis = 1) # make each note x times longer
    p.logging.info(f'after making each note {tile_time_steps = } times longer: {chorale.shape = }')

    chorale = np.repeat(chorale, voices / chorale.shape[0], axis = 0) # make 4 times as many voices
    p.logging.info(f'after making more voices: {chorale.shape = }')
    chorale = np.concatenate((chorale, pad8), axis = 1) # pad some zeros at the end
    p.logging.info(f'after adding all voices and some zeros: {chorale.shape = }')

    # generate a random mask. Input is mask_shape, zero_ratio, one_ratio. 
    # this section occasionally fails because there are a lot of calculations, and I can't seem to get it right.
    mask_zeros = rng.integers(low=zeros_low, high = zeros_high, size = None) # how many zeros in the mask
    mask_ones = rng.integers(low=ones_low, high = ones_high, size = None) # how many ones in the mask
    p.logging.info(f'{mask_zeros = }, {mask_ones = }')
    mask_length = rng.integers(low = mask_length_low, high = mask_length_high, size = None) # how many consecutive notes to mask at a time, could get repetitive
    mask = stretch.shaded_mask((voices, mask_length), mask_zeros,  mask_ones) # build a mask for me, different 0:1 ratio, and length each time  
    p.logging.info(f'{mask.shape = }')

    # make sections an octave or two higher or lower - twice in batches
    chorale = stretch.octave_shift(stretch.octave_shift(chorale, stretch_octaves_percent_up[0], 
            stretch_octaves_duration_up[0], OCTAVE_SIZE), stretch_octaves_percent_up[1], stretch_octaves_duration_up[1], OCTAVE_SIZE)
    chorale = stretch.octave_shift(stretch.octave_shift(chorale, stretch_octaves_percent_down[0], 
            stretch_octaves_duration_down[0], -OCTAVE_SIZE), stretch_octaves_percent_up[1], stretch_octaves_duration_down[1], -OCTAVE_SIZE) 

    # Arpeggiate the chorale by poking holes in it with the masks prepared earlier
    chorale = stretch.arpeggiate(chorale, mask, 1, bass_bump = False) 
    p.logging.info(f'After masking to put random holes in the data: {chorale.shape = }')

    # add a short unarpeggiated section
    # copy the last few time_steps and slow them down by x times
    # add the slow start 20% of the time, except the first time through
    if (i != 0) and (rng.choice([True, False], p = ([.2, .8]))):
        slow_start = np.repeat(chorale[:,slow_time_steps:], slot_repeat, axis = 1)
        p.logging.info(f'{slow_start.shape = }, {chorale.shape = }, {pad16.shape = }')
        chorale = np.concatenate((slow_start, chorale), axis = 1)
    
    slow_end = np.repeat(chorale[:,slow_time_steps:], slot_repeat, axis = 1)
    p.logging.info(f'{slow_end.shape = }, {chorale.shape = }, {pad16.shape = }')
    chorale = np.concatenate((chorale, slow_end), axis = 1)
    
    # add this verse, bridge, & ending to the building collection of same
    concat_chorale =  np.concatenate((concat_chorale, chorale), axis = 1) 

    # how long is the current chorale after all the additions
    current_elapsed = round(chorale.shape[1] / 4, 0)  # assuming t 60 in the csound file
    p.logging.info(f'{current_elapsed = }')
    elapsed_time = elapsed_time + current_elapsed
    p.logging.info(f'{elapsed_time = }')
    # no faster than quarter note = 80 to 120 beats per minute
    new_tempo = min(rng.integers(assigned_tempo_minimum, high = assigned_tempo_maximum, size = None), current_elapsed * 5)
    
    # new_tempo = rng.integers(assigned_tempo_minimum, high = assigned_tempo_maximum, size = None)
    p.logging.info(f'{new_tempo = }')
    tempos = tempos + str(new_tempo) + ' ' + str(elapsed_time) + ' '  + str(new_tempo) + ' ' + str(elapsed_time) + ' ' 
    
    p.logging.info(f'{tempos = }')
    last_time_step[i] = concat_chorale.shape[1]

concat_chorale =  np.concatenate((concat_chorale, pad16), axis = 1) 
tempos = tempos + str(new_tempo)

shaded_mask building. total_size = 304, (zero_ratio + one_ratio) * (total_size / total_ratio) = 304.0


In [88]:
np.save('balloon_drum_percussion', concat_chorale)

In [79]:
# reload(stretch)
csound = True
convolv = True
limit = 0
print(f'{len(voice_array[0]) = }')
print(f'{concat_chorale.shape = }')

challenging_steps = np.empty((0,4),dtype=int) # no challenging steps yet.

csound_params = {'piece': concat_chorale, 'challenging_steps': challenging_steps, 
                 'upsample': ([254, 255, 0, 1, 2]), 
                 'min_delay': 0, 'tpq': 1.0, 'log': LOGNAME, 
                 'csd_file': CSD_FILE, 'zfactor': 95, 'voice_array': voice_array, 'octave_size': OCTAVE_SIZE} 

pfields = stretch.piano_roll_to_pfields(csound_params) 
pfields.sort() 

len(voice_array[0]) = 12
concat_chorale.shape = (16, 400)


In [80]:
# voices to use in the orchestra:
# finger piano 1, bass fp 2, balloon drum low 3, med 4, high 5, bass flute 6, oboe 7, clarinet 8, bassoon 9, french horn 10
# 1    2   3   4    5   6   7   8    9    10
# fnp bfp bdl bdm  bdh bflt obo clar bass frn
#                 S   A   T   B   S   A   T   B   S   A   T   B
collection_of_voices = [[1, 2, 3, 4, 5]]
print(f'{last_time_step.shape = }, {len(collection_of_voices) = }\n{last_time_step = }\n{collection_of_voices = }')

# colums in pfields at this point:
# 0  1      2         3         4     5       6      7       8              9       10        11        
# 1, start, duration, velocity, tone, octave, voice, stereo, left envelope, glide1, upsample, right_envelope,
# 12         13         14      15       16       17         
# 2nd glied, 3rd glide, volume, channel, density, time_step

# pfields = np.array(pfields)    
# print(f'{pfields.shape = }')             
# fix the envelope of the long lasting notes
def update_row(row):
    if row[2] > 1.4:
        row[8] = rng.choice([26, 19, 1, 8])
        row[11] = rng.choice([26, 19, 1, 8])
    elif row[2] > 0.6:
        row[8] = rng.choice([17, 17, 1, 8])
        row[11] = rng.choice([17, 17, 1, 8])
    elif row[2] > 0.4:
        row[8] = 8
        row[11] = 8
    return row

new_pfields = []
rows_in_pfields = len(pfields)
rows_in_collection = 0
not_in_collection = 0
this_collection = 0
this_last_time_step = 0
print(f'{this_collection = }, {last_time_step[this_collection] = }, {len(collection_of_voices) = }')
print(f'{collection_of_voices[this_collection] = }, {this_last_time_step = }')            

for row in pfields:
    if row[6] in collection_of_voices[this_collection]:
        # new_row = update_row(row) # no longer updating the envelopes here
        new_row = row
        new_pfields.append(new_row)
        rows_in_collection += 1
    else:
        not_in_collection += 1
    if row[17] > last_time_step[this_last_time_step]: 
        # print(f'{row[17] = }, {this_collection = }, {last_time_step[this_collection] = }')
        # print(f'{this_last_time_step = }, {last_time_step[this_last_time_step] = }')
        this_last_time_step += 1
        this_last_time_step %= len(last_time_step) 
        this_collection += 1
        this_collection %= len(collection_of_voices)

print(f'\n{not_in_collection = }, {rows_in_collection = }')
print(f'{len(pfields) = }')
print(f'{len(new_pfields) = }')

# new_pfields = np.array([row for row in pfields if ((row[6] in collection[this_collection]) & (row[7] < last_time_step[this_collection]))])          

last_time_step.shape = (1,), len(collection_of_voices) = 1
last_time_step = array([398])
collection_of_voices = [[1, 2, 3, 4, 5]]
this_collection = 0, last_time_step[this_collection] = 398, len(collection_of_voices) = 1
collection_of_voices[this_collection] = [1, 2, 3, 4, 5], this_last_time_step = 0

not_in_collection = 0, rows_in_collection = 1422
len(pfields) = 1422
len(new_pfields) = 1422


In [81]:
csd_content, lines = p.load_csd(CSD_FILE, strip_f0 = True) # load the template csd file, discarding unnecessary lines f0, t0 

new_pfields = stretch.process_pfields_to_csd_file(csd_content, new_pfields, 
                  path_to_output_csd = 'new_output.csd', limit = limit, tempos = tempos) 

if csound:
    os.system(f'csound new_output.csd -Ox{LOGNAME}') # give it a log file to save csound messages to. Can get big.
    os.system(f"egrep 'invalid|replacing|range|error|cannot|rtevent|overall' x{LOGNAME}") # inspect the log for important information

limit = 98.60760480375953
last start time was at 97.5
	 number of samples out of range:       86        0
	 number of samples out of range:        0        3
	 number of samples out of range:       38        0
	 number of samples out of range:        0        1
	 number of samples out of range:        0        7
end of score.		   overall amps:  49306.0  35531.0
	   overall samples out of range:      124       11
0 errors in performance


In [82]:
# convolve with the impulse response fle from Teatro Alcorcon in Madrid from Angelo Farina
if convolv: os.system(f'../coconet-pytorch/coconet-pytorch-csound/trim.sh ball2 {version}')

p.logging.info(f'Completed {version = }')

/home/prent/Music/sflib
total 6.1G
-rw-r--r--. 1 prent prent 6.1M Aug 26 14:55 ball2.eps
-rw-r--r--. 1 prent prent  15M Aug 26 14:55 ball2.wav
-rw-r--r--. 1 prent prent 6.9M Aug 26 14:54 ball2-t19.wav
-rw-r--r--. 1 prent prent 404M Aug 26 14:54 ball2a-c.wav


export SFDIR="/home/prent/Music/sflib"
echo $SFDIR
ls $SFDIR -lth | head -n 5
csound -U sndinfo $SFDIR/"$1".wav
0dBFS level = 32768.0
--Csound version 6.16 (double samples) Aug 10 2021
[commit: none]
libsndfile-1.0.31
util sndinfo:
/home/prent/Music/sflib/ball2.wav:
	srate 44100, stereo, 24 bit WAV, 58.015 seconds
	(2558472 sample frames)
end of score.		   overall amps:      0.0
	   overall samples out of range:        0
0 errors in performance
Elapsed time at end of performance: real: 0.000s, CPU: 0.000s
csound "$1"c.csd 
0dBFS level = 32768.0
--Csound version 6.16 (double samples) Aug 10 2021
[commit: none]
libsndfile-1.0.31
UnifiedCSD:  ball2c.csd
STARTING FILE
Creating options
Creating orchestra
closing tag
Creating score
rtaudio: pulseaudio module enabled
rtmidi: ALSA Raw MIDI module enabled
sorting score ...
	... done
graphics suppressed, ascii substituted
0dBFS level = 32768.0
orch now loaded
audio buffered in 256 sample-frame blocks
writing 1536-byte blks of 24bit ints to /home

In [84]:
# Explore the pfields using pandas
pfield_columns = ['instrument', 'start_time', 'duration', 'velocity', 'tone', 'octave','voice', 'stereo', 'envelope',
                                            'gliss', 'upsample', 'renv', '2gliss', '3gliss', 'volume', 'channel', 'density', 'time_step']
pfields_df = pd.DataFrame(new_pfields, columns = pfield_columns)
pfields_df.describe()

,instrument,start_time,duration,velocity,tone,octave,voice,stereo,envelope,gliss,upsample,renv,2gliss,3gliss,volume,channel,density,time_step
count,1422.0,1422.000000,1422.000000,1422.000000,1422.000000,1422.000000,1422.000000,1422.000000,1422.000000,1422.0,1422.000000,1422.000000,1422.0,1422.0,1422.000000,1422.0,1422.000000,1422.000000
mean,1.0,46.393389,0.527391,63.139944,106.997890,3.554149,1.962729,7.932489,2.663150,0.0,102.779887,2.650492,0.0,0.0,27.948136,1.0,-4.807665,186.280591
std,0.0,26.841204,0.600223,1.995624,56.612906,0.901688,1.228624,4.257503,3.065052,0.0,124.336562,3.061566,0.0,0.0,1.469814,0.0,3.052317,107.945366
min,1.0,0.000000,0.350000,59.000000,0.000000,2.000000,1.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,25.000000,1.0,-12.600000,0.000000
25%,1.0,23.010226,0.350000,61.000000,38.000000,3.000000,1.000000,5.000000,0.000000,0.0,1.000000,0.000000,0.0,0.0,26.750000,1.0,-7.000000,93.250000
50%,1.0,46.627035,0.350000,63.000000,90.000000,4.000000,1.000000,8.000000,1.000000,0.0,2.000000,1.000000,0.0,0.0,28.250000,1.0,-4.500000,187.000000
75%,1.0,69.689880,0.600000,65.000000,146.000000,4.000000,3.000000,11.000000,2.000000,0.0,254.000000,2.000000,0.0,0.0,29.250000,1.0,-2.300000,278.750000
max,1.0,97.507605,6.100000,67.000000,198.000000,6.000000,5.000000,15.000000,8.000000,0.0,255.000000,8.000000,0.0,0.0,30.000000,1.0,-0.100000,393.000000


In [85]:
display(pfields_df.head(50))

,instrument,start_time,duration,velocity,tone,octave,voice,stereo,envelope,gliss,upsample,renv,2gliss,3gliss,volume,channel,density,time_step
0,1,0.000000,0.35,64,179,4,4,1,2,0.0,2,8,0.0,0.0,29.75,1,-0.1,0
1,1,0.004804,0.35,64,179,3,1,9,8,0.0,255,2,0.0,0.0,29.75,1,-0.1,0
2,1,0.006777,0.85,66,34,4,1,11,0,0.0,1,0,0.0,0.0,30.00,1,-0.1,2
3,1,0.229780,0.35,65,90,3,2,9,2,0.0,254,8,0.0,0.0,30.00,1,-0.1,1
4,1,0.251554,0.35,66,90,3,1,8,1,0.0,0,2,0.0,0.0,29.75,1,-0.1,1
5,1,0.254898,0.60,67,34,3,1,3,8,0.0,254,2,0.0,0.0,30.00,1,-0.1,2
6,1,0.258072,0.60,66,138,2,1,3,1,0.0,1,1,0.0,0.0,29.75,1,-0.1,2
7,1,0.479480,0.35,66,138,5,3,8,0,0.0,255,2,0.0,0.0,30.00,1,-0.1,2
8,1,0.501416,0.35,65,138,4,3,15,1,0.0,254,1,0.0,0.0,30.00,1,-0.1,2
9,1,0.503631,0.35,65,34,3,1,5,0,0.0,2,0,0.0,0.0,30.00,1,-0.1,2


In [86]:
# list the value_counts for one column 
# important_columns = ['velocity','tone','octave', 'voice', 'envelope', 'upsample', 'volume', 'channel', 'density']
important_columns = ['voice']
print(f'value_counts for {important_columns = }')
for column in pfield_columns:
    if column in important_columns: 
        print(f'{column = }')
        print(f'{pfields_df[column].value_counts()}')
     

value_counts for important_columns = ['voice']
column = 'voice'
1    744
2    253
3    249
5     90
4     86
Name: voice, dtype: int64
